In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import re
import numpy as np

train_df = pd.read_csv('train_text.csv')
train_df2 = pd.read_csv('test_text.csv')
train_label = pd.read_csv('test1.csv')
train_answer = pd.read_csv('answer_train.csv')
lb = LabelEncoder()
train_answer['label'] = lb.fit_transform(train_answer['label'])
train_label['label'] = lb.transform(train_label['label'])
train_label.columns = ['id', 'label']
train_df2 = pd.merge(train_df2, train_label, on='id', how='left')
train_df = pd.concat([train_df, train_df2], axis=0)
test_df = pd.read_csv('test_text2.csv')

In [22]:
# preprocess
pattern = re.compile("[^\u4e00-\u9fa5]")
def preprocess(x):
    text = str(x['text'])
    text = re.sub(pattern, '', text)
    text = ''.join(text.split())
    title = x['id'].split('.')[0][6:].replace('_', '')
    if '无访问权限' in text or len(text) < 20:
        return title
    elif len(text) > 10000:
        return title + text[:10000]
    else:
        return title + text
    
train_df['text_new'] = train_df.apply(lambda x: preprocess(x), axis=1)
test_df['text_new'] = test_df.apply(lambda x: preprocess(x), axis=1)

train_set, valid_set = train_test_split(train_df, stratify=train_df['label'], test_size=0.2, random_state=2020)
train_set[['label', 'text_new']].to_csv('bert/data_both/train.tsv', sep='\t', index=False, header=None)
valid_set[['label', 'text_new']].to_csv('bert/data_both/dev.tsv', sep='\t', index=False, header=None)

In [2]:
def preprocess2(x):
    text = str(x['text'])
    text = re.sub(pattern, '', text)
    text = ''.join(text.split())
    title = re.sub(pattern, '', x['id'])
    if len(title) == 0:
        if len(text) > 10000:
            return text[:10000]
        else:
            return text        
    else:
        title = x['id'].split('.')[0][6:].replace('_', '')
        if '无访问权限' in text or len(text) < 20:
            return title
        elif len(text) > 10000:
            return title + text[:10000]
        else:
            return title + text
test_df['text_new'] = test_df.apply(lambda x: preprocess2(x), axis=1)

In [4]:
test_df['label'] = 0
test_df[['label', 'text_new']].to_csv('bert/data_both/test.tsv', sep='\t', index=False, header=None)

In [ ]:
# predict
test_predict = pd.read_csv('bert/output_both/test_results.tsv', sep='\t', header=None)
test_label = lb.inverse_transform([np.argmax(i) for i in test_predict.values])
test_df['label'] = test_label

In [17]:
def choose_label(x):
    if x['label_y'] != '0':
        return x['label_y']
    else:
        return x['label_x']

In [31]:
# combine answer
test_df_content = pd.read_csv('content_final.csv')
test_df_title = pd.read_csv('title_final.csv')
test = pd.concat([test_df_content, test_df_title], axis=0)

In [36]:
result = pd.merge(test_df, test, on='id', how='left')
result['label_y'] = result['label_y'].fillna('0')
result['label'] = result.apply(choose_label, axis=1)
result['filename'] = result['id']
result[['filename', 'label']].to_csv('submit_final_5.csv', index=False)